# Generate position, trades and dividends `dfs` from ibkr history
- Go to `Performance & Reports` -> `Custom Statements` -> `history`. 
- Choose appropriate custom date range. Max is 365 days
- Remove account number from the CSV file
- recast the file name to `<MARKET>_<DATE_FROM>_<DATE_TO>.csv`. e.g. `SNP_20230102_20231229.csv`
- move the files to `<root>\data\raw` folder
- run this notebook by selecting the right `MARKET`

## Note
`history` is a standard activity report with the following selected:
 - Open Positions
 - Option Exercises/Assignments
 - Trades
 - Dividends

In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util, LimitOrder, IB
util.startLoop()

In [2]:
# Set the root
from from_root import from_root
ROOT = from_root()

# Imports 
from utils import Vars

_vars = Vars(MARKET)
PORT = port = _vars.PORT
OPT_COLS = _vars.OPT_COLS[0]

In [3]:
import numpy as np
import pandas as pd
from loguru import logger

from utils import trade_extracts

pd.options.display.max_columns =None

ib_reports = trade_extracts(MARKET)

# Analysis
### So, how much money have you made trading in IBKR so far?
 - Check df_trades
 - Remove secType == 'OPT' and code having "A" for Assignments

In [22]:
df = ib_reports['trades']
cond = (df.secType == 'OPT') & df.code.str.contains('A')

net_amt = df[~cond].pnl.sum()

f"Made {'INR' if MARKET == 'NSE' else 'US$'} {net_amt:,.2f} in {MARKET} from {df.time.min().date():%d-%b-%Y} to {df.time.max().date():%d-%b-%Y}"

'Made INR 3,442,536.96 in NSE from 19-Jul-2017 to 15-Feb-2024'